# Maquinito 15 - Entre versos, estrellas y embeddings

# Clave de Cohere

In [ ]:
COHERE_API_KEY = "" # @param {type:"string"}


# Inicialización tróspida de Colab

In [ ]:
!pip install numpy==1.24.4 --force

# Carga de librerías, importación de datos, preparación del sistema

In [ ]:
from IPython.display import clear_output

!pip install annoy
!pip install cohere
!pip install umap-learn

clear_output()

In [ ]:
import cohere
import numpy as np
import pandas as pd
import requests
import umap

from annoy import AnnoyIndex
from google.colab import userdata
from pickle import loads

def get_url_content(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(file="ERROR: no existe el fichero {url}")
        print(response)
        return None
    return response.content

def get_nearest_verses_by_vector(dataframe, search_index, v, cuantos=5):
    similar_item_ids = search_index.get_nns_by_vector(v, cuantos, include_distances=True)
    verses = dataframe.iloc[similar_item_ids[0]]["text"].tolist()
    labels = dataframe.iloc[similar_item_ids[0]]["label_texto"].tolist()
    nearest = list(zip(verses, labels))
    return nearest

In [ ]:
# Lee los versos originales y sus posiciones en el espacio 3D
df = pd.read_csv(f"https://both.rocks/maquinito-15/upniverso_data.csv")


# Lee las categorias de los versos
df_cat = pd.read_csv(f"https://both.rocks/maquinito-15/upniverso_data_categorias.csv", header=None, names=['label', 'label_texto'])


# Mezcla ambos ficheros
df = pd.merge(df, df_cat, on='label', how='left')


# Lee los parámetros de la función que transforma 768 dimensiones en 3
umap_content = get_url_content('https://both.rocks/maquinito-15/upniverso_umap.pkl')
reducer = loads(umap_content)


# Lee los índices para que las búsquedas sean más rápidas
index_content = get_url_content('https://both.rocks/maquinito-15/upniverso_index.ann')
with open('upniverso_index.ann', 'wb') as f:
    f.write(index_content)

search_index = AnnoyIndex(3, "euclidean")
search_index.load('upniverso_index.ann')


# Inicializa la librería de Cohere
try:
  co = cohere.Client(userdata.get('COHERE_API_KEY'))
except:
  co = cohere.Client(COHERE_API_KEY)

# Manos a la obra

In [ ]:
# @title Versos más cercanos a un verso

verso = "El compositor Robert Schumann creó una melodía a partir del nombre de su esposa Clara, transformando las letras en tonos musicales." # @param {type:"string"}

response = co.embed(texts=[verso], model="embed-multilingual-v2.0")
embeddings_768 = response.embeddings[0]
embeddings_3 = reducer.transform([embeddings_768])[0]

nearest_verses = get_nearest_verses_by_vector(df, search_index, embeddings_3, cuantos=3)

print(verso)
print(embeddings_3,"\n")
for v,c in nearest_verses:
    print(f"  - {v}\n    {c}\n")

In [ ]:
# @title Versos más cercanos a un punto

# @markdown *x* varía entre -2 y 7<br>
# @markdown *y* varía entre 1 y 9<br>
# @markdown *z* varía entre 3 y 11

x = 0.1713776 # @param {type:"number"}
y = 2.8129175 # @param {type:"number"}
z = 7.144725 # @param {type:"number"}

embeddings_3 = [x,y,z]

nearest_verses = get_nearest_verses_by_vector(df, search_index, embeddings_3)

for v,c in nearest_verses:
    print(f"  - {v}\n    {c}\n")